In [1]:
%load_ext autoreload
%autoreload 2

import os
import datetime

import numpy as np
import pandas as pd

import torch
import torch.multiprocessing as mp
from torch.nn import functional as F
from torchvision.utils import make_grid
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     
from swadist.utils import spawn_fn
from swadist.data import get_dataloaders
from swadist.train import Trainer
from swadist.optim import LinearPolyLR
from swadist.models import ResNet

# mp.spawn may throw an error without this
os.environ['MKL_THREADING_LAYER'] = 'GNU'

cuda = torch.cuda.is_available()
device = 'cuda' if cuda else 'cpu'
print(f'Using {device}')

if cuda:
    torch.cuda.empty_cache()

seed = int((datetime.date.today() - datetime.date(2022, 4, 11)).total_seconds())
torch.manual_seed(seed)
print(f'seed: {seed}')

Using cuda
seed: 2246400


### SGD training

We train ResNet-8, using the optimal hyperparameters given in [Shallue et al. 2019](http://arxiv.org/abs/1811.03600) via SGD with Nesterov momentum and a linearly-decreasing learning rate schedule.

In [3]:
%%time

# whether to log training to Tensorboard
log = False

batch_size = 256

# optimizer
lr0, momentum,  = 2**-5., 0.975

# scheduler
alpha, decay_epochs = 0.25, 15

# training epochs
epochs_sgd = 15

# loaders
train_loader, valid_loader = get_dataloaders(dataset="cifar10",
                                             batch_size=batch_size, 
                                             num_workers=4, 
                                             test=False,
                                             pin_memory=cuda)

# keep starting params consistent across runs
torch.manual_seed(seed)

# model
resnet8 = ResNet(in_kernel_size=3, 
                 stack_sizes=[1, 1, 1], 
                 n_classes=10, 
                 batch_norm=False)

# optimizer and scheduler
optimizer = torch.optim.SGD(resnet8.parameters(), 
                            lr=lr0, 
                            momentum=momentum, 
                            nesterov=True)

scheduler = LinearPolyLR(optimizer, 
                         alpha=alpha, 
                         decay_epochs=decay_epochs)

# trainer
trainer = Trainer(resnet8, 
                  train_loader, 
                  valid_loader, 
                  F.cross_entropy, 
                  optimizer, 
                  scheduler=scheduler, 
                  device=device,
                  name='sgd',
                  log=log)

# begin training
trainer(epochs_sgd=epochs_sgd)

Using RandomSampler
Number of training samples: 45000
Number of training batches: 176

Starting 15-epoch training loop...
Random seed: 4991803466732119831

SGD epochs: 15 | Codistillation epochs: 0 | SWA epochs: 0
DistributedDataParallel: False
Stopping accuracy: None

Train epoch: 1 | Metrics (epoch mean): cross_entropy=1.924011 <> acc=0.286610 | Batch (size 200): 176/176 (100%) | Total steps: 176
Validation | cross_entropy=1.663748 <> accuracy=0.402309 | Batch: 20/20 (100%)

Train epoch: 2 | Metrics (epoch mean): cross_entropy=1.462216 <> acc=0.467975 | Batch (size 200): 176/176 (100%) | Total steps: 352
Validation | cross_entropy=1.563266 <> accuracy=0.463971 | Batch: 20/20 (100%)

Train epoch: 3 | Metrics (epoch mean): cross_entropy=1.210520 <> acc=0.565708 | Batch (size 200): 176/176 (100%) | Total steps: 528
Validation | cross_entropy=1.096468 <> accuracy=0.612672 | Batch: 20/20 (100%)

Train epoch: 4 | Metrics (epoch mean): cross_entropy=1.034033 <> acc=0.632164 | Batch (size 20

### Validation accuracy by target class

In [3]:
resnet8.to(device)

classes = np.array(['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck'])

correct = 0
total = 0

correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

labels = []
preds = []

with torch.inference_mode():
    for data in valid_loader:
        images, target = data
        labels.append(target.numpy())
        outputs = resnet8(images.to(device)).cpu()
        _, predictions = torch.max(outputs, 1)
        preds.append(predictions)
        total += target.size(0)
        correct += (predictions == target).sum().item()
        # collect the correct predictions for each class
        for label, prediction in zip(target, predictions):
            if label == prediction:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1

print(f'Validation accuracy: {100 * correct // total}%')
            
# print accuracy for each class
for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print(f'Accuracy for class: {classname:5s} is {accuracy:.1f} %')

labels = pd.Series(np.hstack(labels).astype(int), name="Labels")
preds = pd.Series(np.hstack(preds).astype(int), name="Preds")
df_confusion = pd.crosstab(classes[labels], classes[preds])
df_confusion

Validation accuracy: 73%
Accuracy for class: plane is 77.7 %
Accuracy for class: car   is 83.2 %
Accuracy for class: bird  is 54.5 %
Accuracy for class: cat   is 58.4 %
Accuracy for class: deer  is 68.8 %
Accuracy for class: dog   is 64.8 %
Accuracy for class: frog  is 77.7 %
Accuracy for class: horse is 78.1 %
Accuracy for class: ship  is 88.2 %
Accuracy for class: truck is 86.3 %


col_0,bird,car,cat,deer,dog,frog,horse,plane,ship,truck
row_0,,,,,,,,,,
bird,261,1,28,63,30,27,18,38,6,7
car,0,406,4,2,6,0,3,13,14,40
cat,17,4,303,28,100,21,15,17,6,8
deer,20,1,35,327,22,17,36,12,2,3
dog,16,0,106,14,343,11,30,6,1,2
frog,19,0,28,25,25,379,4,2,4,2
horse,8,0,25,37,22,0,379,6,4,4
plane,14,4,9,8,4,3,7,383,42,19
ship,3,9,6,2,2,3,2,27,469,9


### DDP SGD

In [4]:
%%time

if cuda:
    torch.cuda.empty_cache()

# number of model replicas
world_size = 2

dataloader_kwargs = {
    'dataset': 'cifar10',
    # size of training batches on each rank
    'batch_size': 256 // world_size,
    # cpu processes per rank for data loading
    'num_workers': 2,
    # when true, uses DistributedSampler
    'data_parallel': True,
}

model_kwargs = {
    'n_classes': 10,
    'in_kernel_size': 3,
    'stack_sizes': [1, 1, 1],
    'batch_norm': False,
}

optimizer_kwargs = {
    'lr': 2**-5.,
    'momentum': 0.975,
    'nesterov': True,
}

trainer_kwargs = {
    'name': 'sgd',
    # whether to log training to Tensorboard
    'log': False,
}

train_kwargs = {
    'epochs_sgd': 15,
}

scheduler_kwargs = {
    'alpha': 0.25,
    'decay_epochs': 15,
}

args = (world_size,
        dataloader_kwargs,
        model_kwargs,
        optimizer_kwargs,
        trainer_kwargs,
        train_kwargs,
        scheduler_kwargs,
        None, # swa_scheduler_kwargs
        seed) # seed on rank i = seed + i

# begin training
mp.spawn(spawn_fn, args=args, nprocs=world_size, join=True)

Rank 0: joined process group on device cuda with backend nccl
Rank 0: torch.manual_seed(2246400)
Using DistributedSampler
Number of training samples: 45000
Number of training batches: 176

Rank 0: using DistributedDataParallel

Worker 1/2 starting 15-epoch training loop...
SGD epochs: 15 | Codistillation epochs: 0 | SWA epochs: 0
DistributedDataParallel: True
Stopping accuracy: None

Rank 1: joined process group on device cuda with backend nccl
Rank 1: torch.manual_seed(2246400)
Using DistributedSampler
Number of training samples: 45000
Number of training batches: 176

Rank 1: using DistributedDataParallel

Worker 2/2 starting 15-epoch training loop...
Train epoch: 1 | Metrics (epoch mean): cross_entropy=1.852665 <> acc=0.303848 | Batch (size 100): 176/176 (100%) | Total steps: 176
Validation (batch mean) |  cross_entropy=1.565924 <> accuracy=0.425586 | Batch: 40/40 (100%))

Train epoch: 2 | Metrics (epoch mean): cross_entropy=1.411364 <> acc=0.482702 | Batch (size 100): 176/176 (100%)